### 1. Import needed packages and clone the dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from sklearn.datasets import load_files
from keras.utils import np_utils, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

print('You are using numpy version:', np.__version__)

!git clone https://github.com/Horea94/Fruit-Images-Dataset

You are using numpy version: 1.20.1


fatal: destination path 'Fruit-Images-Dataset' already exists and is not an empty directory.


In [ ]:
#import sklearn
#print(sklearn.__version__)

Set the path directory to the cloned test and train data.
load_dataset() returns the filenames, integer classes and string classes that are stored in file directory.


1.   names_train is a vector that contains the filepath of all images from the training set
2.   names_test is a vector that contains the filepath of all images from the test set
3.   intclass_train is a vector containing the int class values (1-131) of all images from the training set
4.   intclass_test is a vector containing the int class values (1-131) of all images from the test set
5.   stringclass_train is a vector containing the string label of class of all images from the training set
6.   stringclass_test is a vector containing the string label of class of all images from the test set






In [2]:
train_dir = 'C:/Users/crysi/Private Dokumente/Papers & wissenschaftl. Arbeiten/Master/Foundations of Machine Learning/First Project/Pycharm/Fruit-Images-Dataset/Training/'
test_dir = 'C:/Users/crysi/Private Dokumente/Papers & wissenschaftl. Arbeiten/Master/Foundations of Machine Learning/First Project/Pycharm/Fruit-Images-Dataset/Test/'

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files, targets, target_labels
    
names_train, intclass_train, stringclass_train = load_dataset(train_dir)
names_test, intclass_test, stringclass_test = load_dataset(test_dir)

print('Loading complete!')
print('Training set size : ',  names_train.shape[0])
print('Testing set size : ', names_test.shape[0])
print(stringclass_train)

Loading complete!
Training set size :  67692
Testing set size :  22688
['Apple Braeburn' 'Apple Crimson Snow' 'Apple Golden 1' 'Apple Golden 2'
 'Apple Golden 3' 'Apple Granny Smith' 'Apple Pink Lady' 'Apple Red 1'
 'Apple Red 2' 'Apple Red 3' 'Apple Red Delicious' 'Apple Red Yellow 1'
 'Apple Red Yellow 2' 'Apricot' 'Avocado' 'Avocado ripe' 'Banana'
 'Banana Lady Finger' 'Banana Red' 'Beetroot' 'Blueberry' 'Cactus fruit'
 'Cantaloupe 1' 'Cantaloupe 2' 'Carambula' 'Cauliflower' 'Cherry 1'
 'Cherry 2' 'Cherry Rainier' 'Cherry Wax Black' 'Cherry Wax Red'
 'Cherry Wax Yellow' 'Chestnut' 'Clementine' 'Cocos' 'Corn' 'Corn Husk'
 'Cucumber Ripe' 'Cucumber Ripe 2' 'Dates' 'Eggplant' 'Fig' 'Ginger Root'
 'Granadilla' 'Grape Blue' 'Grape Pink' 'Grape White' 'Grape White 2'
 'Grape White 3' 'Grape White 4' 'Grapefruit Pink' 'Grapefruit White'
 'Guava' 'Hazelnut' 'Huckleberry' 'Kaki' 'Kiwi' 'Kohlrabi' 'Kumquats'
 'Lemon' 'Lemon Meyer' 'Limes' 'Lychee' 'Mandarine' 'Mango' 'Mango Red'
 'Mangostan' 

Show distribution of images to the different classes.

In [ ]:
(intclass, counts) = np.unique(intclass_train, return_counts=True)
plt.bar(intclass, counts)
plt.title('distribution of full training and test data')
full_train_frequencies = np.asarray((intclass, counts)).T

(intclass, counts) = np.unique(intclass_test, return_counts=True)
plt.bar(intclass, counts)
full_test_frequencies = np.asarray((intclass, counts)).T

Show example file paths and their class label

In [ ]:
i = 0
while i < 3:
  print('Path to image : ', names_train[i])
  print('Class label of image: ', intclass_train[i])
  i+=1

In [3]:
n_classes = len(np.unique(intclass_test))
print('Training set size :',  names_train.shape[0])
print('Testing set size :', names_test.shape[0])
print('Number of unique classes:', n_classes)

Training set size : 67692
Testing set size : 22688
Number of unique classes: 131


Categorized matrix (0 or 1) for class of each image.


1.   train_hot_class is a list containing (length of train images) arrays containing a list each (131 elements) where the i th element is 1, when the image is classified as intclass i (otherwise 0)
2.   test_hot_class is a list containing (length of test images) arrays containing a list each (131 elements) where the i th element is 1, when the image is classified as intclass i (otherwise 0)



In [4]:
train_hot_class = np_utils.to_categorical(intclass_train, n_classes)
test_hot_class = np_utils.to_categorical(intclass_test, n_classes)
train_hot_class[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

Change name of image to actual pixel array.
The _images_array are the inputs (100x100 pixels with 3 color channels).


In [5]:
def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

train_images_array = np.array(convert_image_to_array(names_train))
print('Training set shape : ', train_images_array.shape)

test_images_array = np.array(convert_image_to_array(names_test))
print('Test set shape : ', test_images_array.shape)

print('1st training image shape ',train_images_array[0].shape)

Training set shape :  (67692, 100, 100, 3)
Test set shape :  (22688, 100, 100, 3)
1st training image shape  (100, 100, 3)


Pixel arrays of one image (100x100 pixels, 3 color channels).

In [ ]:
print('1st training image as array',test_images_array[0])

Rescale pixel values from 0-255 range to 0-1.


1.   train_images_array is an array containing the normalized pixel values of the train images.
2.   test_images_array is an array containing the normalized pixel values of the test images.
3.   valid_images_array is an array containing the normalized pixel values of the validation images.



In [6]:
train_images_array = train_images_array.astype('float32')/255
test_images_array = test_images_array.astype('float32')/255

Split the train datasets into a train dataset and a validation dataset (~75/25).


1.   names_train is a list of names of the first half of the train dataset (actual training dataset)
2.   names_valid is a list of names of the second half of the train dataset (validation dataset)
3.   train_hot_class is the first half of train_hot_class (actual training categorial classes)
4.   valid_hot_class is the second half of train_hot_class (validation categorial classes)



In [7]:
def split(a, n):
    o, m = divmod(len(a), n)
    return (a[i*o+min(i, m):(i+1)*o+min(i+1, m)] for i in range(n))

In [8]:
def compute_CV_NeuralNetwork(x_data, y_data, k = 5):
    rand = random.sample(range(x_data.shape[0]), x_data.shape[0])
    k_split_rand = list(split(rand, k))
    for i in range(k):
        seq = k_split_rand[i]
        mask = np.ones(x_data.shape[0],dtype=bool)
        mask[seq]=0
        #x_valid = x_data[seq]
        #y_valid = y_data[seq]
        #x_train = x_data[mask]
        #y_train = y_data[mask]
        network = build_NeuralNetwork()
        print('Model built.')
        #checkpointer = ModelCheckpoint(filepath = 'CVann1.hdf5', verbose = 0, save_best_only = True)

        network.fit(x_data[mask], y_data[mask],
            batch_size = 32, #32
            epochs = 20, #15
            validation_data = (x_data[seq], y_data[seq]),
            verbose = 2,
            #callbacks=[checkpointer],
            shuffle = True)
        network_name = 'model1CV'+str(i)+'.h5'
        network.save(network_name)
        train_loss, train_acc = network.evaluate(x_data[mask], y_data[mask], verbose=0)
        print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
        valid_loss, valid_acc = network.evaluate(x_data[seq], y_data[seq], verbose=0)
        print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))

In [9]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(1))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.5))

    #hiddenlayer 3
    #model.add(Dense(1000)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.5))

    #hiddenlayer 4
    #model.add(Dense(1000)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.5))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [10]:
compute_CV_NeuralNetwork(train_images_array, train_hot_class, k = 2)

Compiled!
Model built.
Epoch 1/20
1058/1058 - 22s - loss: 4.7038 - accuracy: 0.0232 - val_loss: 4.5643 - val_accuracy: 0.0264 - 22s/epoch - 21ms/step
Epoch 2/20
1058/1058 - 5s - loss: 4.4995 - accuracy: 0.0281 - val_loss: 4.4464 - val_accuracy: 0.0358 - 5s/epoch - 5ms/step
Epoch 3/20
1058/1058 - 4s - loss: 4.3990 - accuracy: 0.0341 - val_loss: 4.7287 - val_accuracy: 0.0108 - 4s/epoch - 4ms/step
Epoch 4/20
1058/1058 - 4s - loss: 4.3321 - accuracy: 0.0361 - val_loss: 4.4000 - val_accuracy: 0.0324 - 4s/epoch - 4ms/step
Epoch 5/20
1058/1058 - 4s - loss: 4.2791 - accuracy: 0.0355 - val_loss: 4.7444 - val_accuracy: 0.0193 - 4s/epoch - 4ms/step
Epoch 6/20
1058/1058 - 4s - loss: 4.2380 - accuracy: 0.0377 - val_loss: 4.8672 - val_accuracy: 0.0139 - 4s/epoch - 4ms/step
Epoch 7/20
1058/1058 - 4s - loss: 4.2101 - accuracy: 0.0395 - val_loss: 4.4833 - val_accuracy: 0.0272 - 4s/epoch - 4ms/step
Epoch 8/20
1058/1058 - 4s - loss: 4.1810 - accuracy: 0.0386 - val_loss: 4.1429 - val_accuracy: 0.0471 - 4s

MemoryError: Unable to allocate 3.78 GiB for an array with shape (33846, 100, 100, 3) and data type float32

In [ ]:
new_model = tf.keras.models.load_model('Newann1CV1.h5')
loss, acc = new_model.evaluate(test_images_array, test_hot_class, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

In [ ]:
from sklearn.model_selection import train_test_split
train_images_array, valid1_images_array, train1_hot_class, valid1_hot_class = train_test_split(train_images_array, train_hot_class,    test_size = 0.1, random_state=12)
print('Vaildation structure of pixels : ', valid1_images_array.shape)
print('Vaildation structure of categorial classes :', valid1_hot_class.shape)
print('Train structure of images: ', train1_images_array.shape)
print('Train structure of categorial classes : ', train1_hot_class.shape)
print('------')
#print('Test structure of images: ', test_images_array.shape)
#print('Test structure of categorial classes : ', test_hot_class.shape)
#valid1_images_array, test1_images_array, valid1_hot_class, test1_hot_class = train_test_split(valid1_images_array, valid1_hot_class, test_size = 0.5, random_state=12)
print('Train structure of images: ', train1_images_array.shape)
print('Train structure of categorial classes : ', train1_hot_class.shape)
print('Vaildation structure of pixels : ', valid1_images_array.shape)
print('Vaildation structure of categorial classes :', valid1_hot_class.shape)
print('Test structure of images: ', test1_images_array.shape)
print('Test structure of categorial classes : ', test1_hot_class.shape)

Show 10 images.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize =(30,5))
for i in range(10):
    ax = fig.add_subplot(2,5,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(train1_images_array[i]))
    print(names_train[i])

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize =(30,5))
for i in range(10):
    ax = fig.add_subplot(2,5,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(test_images_array[i]))
    print(names_test[i])

build neural network model

In [ ]:
network = build_NeuralNetwork()
network.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath = 'ann3.hdf5', verbose = 2, save_best_only = True)
#filename='firstANN.csv'
#history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
history = network.fit(train1_images_array, train1_hot_class,
    batch_size = 32, #32
    epochs = 25, #15
    validation_data = (valid1_images_array, valid1_hot_class),
    verbose = 2,
    callbacks=[checkpointer], shuffle = True)
network.save('ann3.h5')

In [ ]:
new_model = tf.keras.models.load_model('ann3.h5')
loss, acc = new_model.evaluate(valid1_images_array, valid1_hot_class, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

In [ ]:
loss, acc = new_model.evaluate(test_images_array, test_hot_class, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))


In [ ]:
model.load_weights('ann.hdf5')
score = model.evaluate(test_images_array, test_hot_class, verbose = 0)
print('\n', 'Test accuracy:', score[1])

In [ ]:
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(train_images_array[10].reshape(1,100,100,1))
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

In [ ]:
history.model.weights

In [ ]:
import matplotlib.pyplot as plt
y_pred = model.predict(test_images_array)

fig = plt.figure(figsize=(32, 30))
for i, idx in enumerate(np.random.choice(test_images_array.shape[0], size = 32, replace = False)):
    ax = fig.add_subplot(8, 6, i + 1, xticks = [], yticks = [])
    ax.imshow(np.squeeze(test_images_array[idx]))
    pred_idx = np.argmax(y_pred[idx])
    true_idx = np.argmax(test_hot_class[idx])
    ax.set_title("{} ({})".format(stringclass_test[pred_idx], stringclass_test[true_idx]),
                 color = ("green" if pred_idx == true_idx else "red"))

Model accuracy

In [ ]:
plt.figure(figsize = (6, 5))
plt.plot(history.history['accuracy'], color = 'r')
plt.plot(history.history['val_accuracy'], color = 'b')
plt.title('Model Accuracy', weight = 'bold', fontsize = 16)
plt.ylabel('accuracy', weight = 'bold', fontsize = 14)
plt.xlabel('epoch', weight = 'bold', fontsize = 14)
plt.ylim(0.4, 1)
plt.xticks(weight = 'bold', fontsize = 12)
plt.yticks(weight = 'bold', fontsize = 12)
plt.legend(['train', 'val'], loc = 'lower right', prop = {'size': 14})
plt.grid(color = 'y', linewidth = '0.5')
plt.show()

Show prediction.

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(1)  
   
 # summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc = 'lower right')
   
 # summarize history for loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc = 'upper right')
plt.show()